[toc]

# GBDT sklearn 源码分析


接下来，我们将研究一下 sklearn 中的 `GradientBoostingRegressor` 的实现

```
class GradientBoostingRegressor(RegressorMixin, BaseGradientBoosting):
```

GradientBoostingRegression 继承自 BaseGradientBoosting 类。BaseGradientBoosting 类的源码在 [`scikit-learn/_gb.py`](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/ensemble/_gb.py#L133) 中

## `_fit_stage`

BaseGradientBoosting类有一个 `_fit_stage` 方法，定义了如何进行一次更新。

```
def _fit_stage(self, i, X, y, raw_predictions, sample_weight, sample_mask,
               random_state, X_idx_sorted, X_csc=None, X_csr=None):
    
    assert sample_mask.dtype == np.bool
    loss = self.loss_
    original_y = y

    raw_predictions_copy = raw_predictions.copy()

    for k in range(loss.K):
        if loss.is_multi_class:
            y = np.array(original_y == k, dtype=np.float64)

        residual = loss.negative_gradient(y, raw_predictions_copy, k=k,
                                          sample_weight=sample_weight)

        # induce regression tree on residuals
        tree = DecisionTreeRegressor(
            criterion=self.criterion,
            splitter='best',
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            min_weight_fraction_leaf=self.min_weight_fraction_leaf,
            min_impurity_decrease=self.min_impurity_decrease,
            min_impurity_split=self.min_impurity_split,
            max_features=self.max_features,
            max_leaf_nodes=self.max_leaf_nodes,
            random_state=random_state,
            ccp_alpha=self.ccp_alpha)

        if self.subsample < 1.0:
            # no inplace multiplication!
            sample_weight = sample_weight * sample_mask.astype(np.float64)

        X = X_csr if X_csr is not None else X
        tree.fit(X, residual, sample_weight=sample_weight,
                 check_input=False, X_idx_sorted=X_idx_sorted)

        loss.update_terminal_regions(
            tree.tree_, X, y, residual, raw_predictions, sample_weight,
            sample_mask, learning_rate=self.learning_rate, k=k)

        self.estimators_[i, k] = tree

    return raw_predictions
```

### loss.K

其中，loss 是一个 LossFuncition 对象，代码位于 [`scikit-learn/_gb_losses.py`](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/ensemble/_gb_losses.py#L17-L17) 中，部分代码如下：

```
class LossFunction(metaclass=ABCMeta):
    """Abstract base class for various loss functions.

    Parameters
    ----------
    n_classes : int
        Number of classes.

    Attributes
    ----------
    K : int
        The number of regression trees to be induced;
        1 for regression and binary classification;
        ``n_classes`` for multi-class classification.
    """

    is_multi_class = False

    def __init__(self, n_classes):
        self.K = n_classes
```

可以看到，.K 是 loss 对象的一个属性，表示有几类。对于回归或二分类问题，K = 1.

因此，`_fit_stage` 函数的主体是一个 for 循环，对**每一类**进行循环。而在循环内部，计算了负梯度

```
residual = loss.negative_gradient(y, raw_predictions_copy, k=k,
                                      sample_weight=sample_weight)
```

然后生成了一个 `DecisionTreeRegressor` 回归树对象，并调用 fit 方法进行学习，其中，target 参数传入的是 residual，也即将负梯度作为目标进行拟合。


```
tree.fit(X, residual, sample_weight=sample_weight,
             check_input=False, X_idx_sorted=X_idx_sorted)
```

这里可以得到结论：
1. 对于回归问题或二分类问题，GBDT 每次迭代拟合一棵树。
2. 而对于多分类问题，假设分类数为 K，那么每次会拟合 K 棵树。这些树的输出最后会用来进行 softmax

### loss.update_terminal_regions

在 `tree.fit` 调用完之后，有一个 `loss.update_terminal_regions` 的操作，这个函数也是在 LossFunction 类定义的，如下：

```
    def update_terminal_regions(self, tree, X, y, residual, raw_predictions,
                                sample_weight, sample_mask,
                                learning_rate=0.1, k=0):
        """Update the terminal regions (=leaves) of the given tree and
        updates the current predictions of the model. Traverses tree
        and invokes template method `_update_terminal_region`.
        """
        
        # compute leaf for each sample in ``X``.
        terminal_regions = tree.apply(X)

        # mask all which are not in sample mask.
        masked_terminal_regions = terminal_regions.copy()
        masked_terminal_regions[~sample_mask] = -1

        # update each leaf (= perform line search)
        for leaf in np.where(tree.children_left == TREE_LEAF)[0]:
            self._update_terminal_region(tree, masked_terminal_regions,
                                         leaf, X, y, residual,
                                         raw_predictions[:, k], sample_weight)

        # update predictions (both in-bag and out-of-bag)
        raw_predictions[:, k] += \
            learning_rate * tree.value[:, 0, 0].take(terminal_regions, axis=0)

    @abstractmethod
    def _update_terminal_region(self, tree, terminal_regions, leaf, X, y,
                                residual, raw_predictions, sample_weight):
        """Template method for updating terminal regions (i.e., leaves)."""

```

根据函数的描述可以看出，update_terminal_regions 有两个作用，一个是修改参数 tree 的叶节点的输出。一个是更新参数 raw_predictions。更新叶节点的输出时，调用了 `self._update_terminal_region`，而更新 raw_predictions 是直接将预测值乘以 learning_rate 加到 raw_predictions上。

而 `_update_terminal_region` 函数是用 abstractmethod 修改的抽象函数，是在子类中定义的。

由于 `GradientBoostingRegressor` 的 Loss 取的是 `LeastSquaresError`，因此我们再看看 `LeastSquaresError` 中的 `update_terminal_region` 函数是如何定义的。

```
def update_terminal_regions(self, tree, X, y, residual, raw_predictions,
                                sample_weight, sample_mask,
                                learning_rate=0.1, k=0):
    """Least squares does not need to update terminal regions.
    But it has to update the predictions.
    Parameters
    ----------
    tree : tree.Tree
        The tree object.
    X : ndarray of shape (n_samples, n_features)
        The data array.
    y : ndarray of shape (n_samples,)
        The target labels.
    residual : ndarray of shape (n_samples,)
        The residuals (usually the negative gradient).
    raw_predictions : ndarray of shape (n_samples, K)
        The raw predictions (i.e. values from the tree leaves) of the
        tree ensemble at iteration ``i - 1``.
    sample_weight : ndarray of shape (n,)
        The weight of each sample.
    sample_mask : ndarray of shape (n,)
        The sample mask to be used.
    learning_rate : float, default=0.1
        Learning rate shrinks the contribution of each tree by
         ``learning_rate``.
    k : int, default=0
        The index of the estimator being updated.
    """
    # update predictions
    raw_predictions[:, k] += learning_rate * tree.predict(X).ravel()

def _update_terminal_region(self, tree, terminal_regions, leaf, X, y,
                            residual, raw_predictions, sample_weight):
    pass
```

根据函数描述可以看到，`LeastSquaresError` 不需要修改叶节点的输出值。也就是说，叶节点的输出值就是 `tree.fit` 得到的输出值，只不过在预测的时候需要乘以 learning_rate 之后再累加到之前的结果中。

从源码中可以看出来，`GraidentBoostingRegressor` 的实现中，只有一次优化，也就是基学习器的优化，

## `_fit_stages`

了解了基学习器是如何学习的，再看看基学习器的结果是如何组合的。BaseGradientBoosting 类的 `_fit_stages` 方法描述了多个基学习器是如何学习的。

```
    def _fit_stages(self, X, y, raw_predictions, sample_weight, random_state,
                    X_val, y_val, sample_weight_val,
                    begin_at_stage=0, monitor=None, X_idx_sorted=None):
        """Iteratively fits the stages.

        For each stage it computes the progress (OOB, train score)
        and delegates to ``_fit_stage``.
        Returns the number of stages fit; might differ from ``n_estimators``
        due to early stopping.
        """
        n_samples = X.shape[0]
        do_oob = self.subsample < 1.0
        sample_mask = np.ones((n_samples, ), dtype=np.bool)
        n_inbag = max(1, int(self.subsample * n_samples))
        loss_ = self.loss_

        if self.verbose:
            verbose_reporter = VerboseReporter(self.verbose)
            verbose_reporter.init(self, begin_at_stage)

        X_csc = csc_matrix(X) if issparse(X) else None
        X_csr = csr_matrix(X) if issparse(X) else None

        if self.n_iter_no_change is not None:
            loss_history = np.full(self.n_iter_no_change, np.inf)
            # We create a generator to get the predictions for X_val after
            # the addition of each successive stage
            y_val_pred_iter = self._staged_raw_predict(X_val)

        # perform boosting iterations
        i = begin_at_stage
        for i in range(begin_at_stage, self.n_estimators):

            # subsampling
            if do_oob:
                sample_mask = _random_sample_mask(n_samples, n_inbag,
                                                  random_state)
                # OOB score before adding this stage
                old_oob_score = loss_(y[~sample_mask],
                                      raw_predictions[~sample_mask],
                                      sample_weight[~sample_mask])

            # fit next stage of trees
            raw_predictions = self._fit_stage(
                i, X, y, raw_predictions, sample_weight, sample_mask,
                random_state, X_idx_sorted, X_csc, X_csr)

            # track deviance (= loss)
            if do_oob:
                self.train_score_[i] = loss_(y[sample_mask],
                                             raw_predictions[sample_mask],
                                             sample_weight[sample_mask])
                self.oob_improvement_[i] = (
                    old_oob_score - loss_(y[~sample_mask],
                                          raw_predictions[~sample_mask],
                                          sample_weight[~sample_mask]))
            else:
                # no need to fancy index w/ no subsampling
                self.train_score_[i] = loss_(y, raw_predictions, sample_weight)

            if self.verbose > 0:
                verbose_reporter.update(i, self)

            if monitor is not None:
                early_stopping = monitor(i, self, locals())
                if early_stopping:
                    break

            # We also provide an early stopping based on the score from
            # validation set (X_val, y_val), if n_iter_no_change is set
            if self.n_iter_no_change is not None:
                # By calling next(y_val_pred_iter), we get the predictions
                # for X_val after the addition of the current stage
                validation_loss = loss_(y_val, next(y_val_pred_iter),
                                        sample_weight_val)

                # Require validation_score to be better (less) than at least
                # one of the last n_iter_no_change evaluations
                if np.any(validation_loss + self.tol < loss_history):
                    loss_history[i % len(loss_history)] = validation_loss
                else:
                    break

        return i + 1
```

从上面的代码可以看出，学习的过程的逻辑也是挺直接的。

主要的内容是一个 for 循环，从 i 从 begin_at_stage 开始遍历到 self.n_estimator。进入 for 循环内部。先判断是否选择 oob，如果是，就进行抽样，并计算此时的袋外误差。

之所有不是从 0 开始而是从 begin_at_stage 开始，是为了支持从某个阶段开始继续训练的功能。

然后再调用上面讨论过的 `_fit_stage` 进行一个基学习器的学习。学习完之后记录当前 score，并记录袋外估计的 loss 减少（如果有 oob 的话）。

最后判断是否需要 early stopping。

## fit

上面的过程实际上已经比较清晰地说明了如何进行基学习器的学习与组合。不过，上面并没有说明如何初始化预测值。根据我们的理论知识，GBDT 对于回归问题是用样本均值来初始化预测值的。因此我们再看看如何初始化代码中是如何进行的。

BaseGradietBoosting 的 fit 函数调用进行了第一个基学习器的初始化并调用了 `_fit_stages` 方法。

不过这个函数代码有点多， 因此就只调我们关心的如何初始化基学习器的部分来看。


```
def fit(self, X, y, sample_weight=None, monitor=None):
    // 这里省略一些代码 ....
    if not self._is_initialized():
        # init state
        self._init_state()

        # fit initial model and initialize raw predictions
        if self.init_ == 'zero':
            raw_predictions = np.zeros(shape=(X.shape[0], self.loss_.K),
                                       dtype=np.float64)
        else:
            # XXX clean this once we have a support_sample_weight tag
            if sample_weight_is_none:
                self.init_.fit(X, y)
            else:
                msg = ("The initial estimator {} does not support sample "
                       "weights.".format(self.init_.__class__.__name__))
                try:
                    self.init_.fit(X, y, sample_weight=sample_weight)
                except TypeError:  # regular estimator without SW support
                    raise ValueError(msg)
                except ValueError as e:
                    if "pass parameters to specific steps of "\
                       "your pipeline using the "\
                       "stepname__parameter" in str(e):  # pipeline
                        raise ValueError(msg) from e
                    else:  # regular estimator whose input checking failed
                        raise

            raw_predictions = \
                self.loss_.get_init_raw_predictions(X, self.init_)

        begin_at_stage = 0

        # The rng state must be preserved if warm_start is True
        self._rng = check_random_state(self.random_state)
```

可以看到，初始化基学习器并进行第一次预测的部分在 

```
raw_predictions = \
                self.loss_.get_init_raw_predictions(X, self.init_)
```

调用了 `self.loss_` 对象的 `get_init_raw_predictions` 方法。

我们看看 `RegressionLossFunction` 的 `get_init_raw_predictions` 方法的内容是什么。`RegressionLossFunction` 继承自 `LossFunction` 类，并且是 `LeastSquaresError` 的父类。

```
def get_init_raw_predictions(self, X, estimator):
    predictions = estimator.predict(X)
    return predictions.reshape(-1, 1).astype(np.float64)
```

可以看到，这个函数接受一个 X 和一个 estimator，并调用了 estimator 的 predict 对象。而在 `fit` 函数中， estimator 参数传入的是 `self.init_`。

而 `self.init_` 又是怎么来的呢？ `self.init_` 是在 `self._init_state()` 的时候进行初始化的。我们可以看到，fit 函数中有这样的内容

```
if not self._is_initialized():
    # init state
    self._init_state()
```

说明如果没有初始化，那么就调用 `self._init_state()` 进行初始化。

我们再进入 `BaseGradientBooting` 的 `_init_state` 方法中看一看它是如何初始化 `self._init` 的。
 
```
def _init_state(self):
    """Initialize model state and allocate model state data structures. """

    self.init_ = self.init
    if self.init_ is None:
        self.init_ = self.loss_.init_estimator()

    self.estimators_ = np.empty((self.n_estimators, self.loss_.K),
                                dtype=np.object)
    self.train_score_ = np.zeros((self.n_estimators,), dtype=np.float64)
    # do oob?
    if self.subsample < 1.0:
        self.oob_improvement_ = np.zeros((self.n_estimators),
                                         dtype=np.float64)
```




最后，我们看看 `init_estimator` 是如何定义的。`init_estimator` 定义在 `LeastSquaresError` 中，我们看看

```
def init_estimator(self):
    return DummyRegressor(strategy='mean')
```